# 🧠 Brain Tumor Detection - Model Training Pipeline

## Complete Deep Learning Training Guide

This notebook contains the complete process to train a Convolutional Neural Network (CNN) for brain tumor detection from MRI scans.

### Dataset Information
- **Source**: [Kaggle Brain Tumor Detection Dataset](https://www.kaggle.com/datasets/ahmedhamada0/brain-tumor-detection)
- **Total Images**: 7,000+ MRI scans
- **Classes**: 4 tumor types (Glioma, Meningioma, No Tumor, Pituitary)
- **Image Size**: 224x224 pixels
- **Format**: JPG, PNG

### Model Performance Target
- **Accuracy**: 97%+
- **Precision**: 96%+
- **Recall**: 97%+
- **F1-Score**: 96%+

### Training Environment
- Python 3.8+
- TensorFlow 2.10.0
- Keras 2.10.0
- CUDA (Optional for GPU acceleration)

---

## Section 1: Import Required Libraries and Dependencies

In [ ]:
# Import Core Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import cv2
from pathlib import Path
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

# Import TensorFlow and Keras
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models, Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import (
    EarlyStopping, 
    ModelCheckpoint, 
    ReduceLROnPlateau,
    TensorBoard
)
from tensorflow.keras.optimizers import Adam

# Import Scikit-learn for metrics
from sklearn.metrics import (
    classification_report,
    confusion_matrix,
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    roc_auc_score,
    roc_curve
)

# Check TensorFlow version and GPU availability
print(f"TensorFlow Version: {tf.__version__}")
print(f"GPU Available: {tf.config.list_physical_devices('GPU')}")
print(f"Python Version: {tf.__version__}")

# Set random seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

## Section 2: Load and Prepare Training Data

### Dataset Path Configuration
Update the `dataset_path` variable to point to your downloaded Kaggle dataset directory.

**Dataset Structure Expected:**
```
brain_tumor_dataset/
├── Training/
│   ├── glioma_tumor/
│   ├── meningioma_tumor/
│   ├── no_tumor/
│   └── pituitary_tumor/
└── Testing/
    ├── glioma_tumor/
    ├── meningioma_tumor/
    ├── no_tumor/
    └── pituitary_tumor/
```

In [ ]:
# Configure dataset paths
# IMPORTANT: Update this path to your downloaded Kaggle dataset
dataset_path = "path/to/brain_tumor_dataset"  # Change this to your dataset location

# Verify dataset exists
if not os.path.exists(dataset_path):
    print(f"⚠️ Dataset path not found: {dataset_path}")
    print("Please download the dataset from: https://www.kaggle.com/datasets/ahmedhamada0/brain-tumor-detection")
else:
    print(f"✓ Dataset found at: {dataset_path}")

# Define image parameters
IMG_SIZE = 224
BATCH_SIZE = 32
EPOCHS = 50

# Classes for classification
CLASSES = ['glioma_tumor', 'meningioma_tumor', 'no_tumor', 'pituitary_tumor']
NUM_CLASSES = len(CLASSES)

print(f"\nImage Size: {IMG_SIZE}x{IMG_SIZE}")
print(f"Batch Size: {BATCH_SIZE}")
print(f"Number of Classes: {NUM_CLASSES}")
print(f"Classes: {CLASSES}")

In [ ]:
# Data Augmentation for Training Data
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2  # 20% validation, 80% training
)

# No augmentation for testing/validation data, only rescaling
test_datagen = ImageDataGenerator(rescale=1./255)

# Load training data from directory
train_data = train_datagen.flow_from_directory(
    os.path.join(dataset_path, 'Training'),
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training',
    seed=42
)

# Load validation data
validation_data = train_datagen.flow_from_directory(
    os.path.join(dataset_path, 'Training'),
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation',
    seed=42
)

# Load testing data
test_data = test_datagen.flow_from_directory(
    os.path.join(dataset_path, 'Testing'),
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

# Display data information
print("Training Data Shape:", train_data.samples)
print("Validation Data Shape:", validation_data.samples)
print("Test Data Shape:", test_data.samples)
print("\nClass Mapping:")
for class_name, index in train_data.class_indices.items():
    print(f"  {index}: {class_name}")

## Section 3: Define the Model Architecture

### CNN Architecture Overview
We'll build a deep convolutional neural network with:
- **Conv2D layers** for feature extraction
- **MaxPooling layers** for dimensionality reduction
- **Dropout layers** for regularization
- **Dense layers** for classification
- **Batch Normalization** for improved training stability

In [ ]:
# Build the CNN Model
model = Sequential([
    # Block 1
    layers.Conv2D(32, (3, 3), activation='relu', padding='same', 
                  input_shape=(IMG_SIZE, IMG_SIZE, 3), name='conv1_1'),
    layers.BatchNormalization(),
    layers.Conv2D(32, (3, 3), activation='relu', padding='same', name='conv1_2'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2), name='pool1'),
    layers.Dropout(0.25),
    
    # Block 2
    layers.Conv2D(64, (3, 3), activation='relu', padding='same', name='conv2_1'),
    layers.BatchNormalization(),
    layers.Conv2D(64, (3, 3), activation='relu', padding='same', name='conv2_2'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2), name='pool2'),
    layers.Dropout(0.25),
    
    # Block 3
    layers.Conv2D(128, (3, 3), activation='relu', padding='same', name='conv3_1'),
    layers.BatchNormalization(),
    layers.Conv2D(128, (3, 3), activation='relu', padding='same', name='conv3_2'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2), name='pool3'),
    layers.Dropout(0.25),
    
    # Block 4
    layers.Conv2D(256, (3, 3), activation='relu', padding='same', name='conv4_1'),
    layers.BatchNormalization(),
    layers.Conv2D(256, (3, 3), activation='relu', padding='same', name='conv4_2'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2), name='pool4'),
    layers.Dropout(0.25),
    
    # Flattening and Dense layers
    layers.Flatten(name='flatten'),
    layers.Dense(512, activation='relu', name='dense1'),
    layers.BatchNormalization(),
    layers.Dropout(0.5),
    layers.Dense(256, activation='relu', name='dense2'),
    layers.BatchNormalization(),
    layers.Dropout(0.5),
    layers.Dense(NUM_CLASSES, activation='softmax', name='output')
])

# Display model summary
print("Model Architecture:")
model.summary()

## Section 4: Compile the Model

### Compilation Details
- **Optimizer**: Adam (adaptive learning rate)
- **Loss Function**: Categorical Crossentropy (for multi-class classification)
- **Metrics**: Accuracy for monitoring model performance

In [ ]:
# Compile the model
optimizer = Adam(learning_rate=0.001)

model.compile(
    optimizer=optimizer,
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

print("Model compiled successfully!")
print(f"Optimizer: {optimizer.__class__.__name__}")
print(f"Loss Function: Categorical Crossentropy")
print(f"Metrics: Accuracy")

## Section 5: Train the Model

### Training Strategy
- **Epochs**: 50 (with early stopping)
- **Batch Size**: 32
- **Early Stopping**: Stop if validation accuracy doesn't improve for 5 epochs
- **Model Checkpoint**: Save best model during training
- **Learning Rate Reduction**: Reduce learning rate if loss plateaus

In [ ]:
# Create callback functions for training
callbacks = [
    # Early stopping to prevent overfitting
    EarlyStopping(
        monitor='val_accuracy',
        patience=5,
        restore_best_weights=True,
        verbose=1
    ),
    
    # Save best model
    ModelCheckpoint(
        'best_brain_tumor_model.h5',
        monitor='val_accuracy',
        save_best_only=True,
        verbose=1
    ),
    
    # Reduce learning rate on plateau
    ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=3,
        min_lr=1e-6,
        verbose=1
    ),
    
    # TensorBoard for visualization
    TensorBoard(log_dir='./logs', histogram_freq=1)
]

# Train the model
print("Starting model training...")
print("This may take several hours depending on your hardware.\n")

history = model.fit(
    train_data,
    epochs=EPOCHS,
    validation_data=validation_data,
    callbacks=callbacks,
    verbose=1,
    steps_per_epoch=train_data.samples // BATCH_SIZE,
    validation_steps=validation_data.samples // BATCH_SIZE
)

print("\n✓ Training completed successfully!")

## Section 6: Evaluate Model Performance

### Evaluation Metrics
- **Accuracy**: Overall correct predictions
- **Precision**: Correct positive predictions out of all positive predictions
- **Recall**: Correct positive predictions out of all actual positives
- **F1-Score**: Harmonic mean of precision and recall
- **Confusion Matrix**: True/False positives and negatives
- **ROC Curve**: Receiver Operating Characteristic curve

In [ ]:
# Plot training history
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Accuracy plot
axes[0].plot(history.history['accuracy'], label='Training Accuracy', linewidth=2)
axes[0].plot(history.history['val_accuracy'], label='Validation Accuracy', linewidth=2)
axes[0].set_title('Model Accuracy Over Epochs', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Epoch', fontsize=12)
axes[0].set_ylabel('Accuracy', fontsize=12)
axes[0].legend(fontsize=10)
axes[0].grid(True, alpha=0.3)

# Loss plot
axes[1].plot(history.history['loss'], label='Training Loss', linewidth=2)
axes[1].plot(history.history['val_loss'], label='Validation Loss', linewidth=2)
axes[1].set_title('Model Loss Over Epochs', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Epoch', fontsize=12)
axes[1].set_ylabel('Loss', fontsize=12)
axes[1].legend(fontsize=10)
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('training_history.png', dpi=300, bbox_inches='tight')
plt.show()

print("Training history plot saved as 'training_history.png'")

In [ ]:
# Evaluate on test data
print("Evaluating model on test set...\n")

# Get predictions
test_loss, test_accuracy = model.evaluate(test_data, verbose=1)

print(f"\n{'='*50}")
print(f"Test Results:")
print(f"{'='*50}")
print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy*100:.2f}%")
print(f"{'='*50}")

# Get predictions for detailed metrics
y_true = []
y_pred = []

for images, labels in test_data:
    predictions = model.predict(images, verbose=0)
    y_true.extend(np.argmax(labels, axis=1))
    y_pred.extend(np.argmax(predictions, axis=1))

# Calculate metrics
accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred, average='weighted', zero_division=0)
recall = recall_score(y_true, y_pred, average='weighted', zero_division=0)
f1 = f1_score(y_true, y_pred, average='weighted', zero_division=0)

print(f"\nDetailed Performance Metrics:")
print(f"{'='*50}")
print(f"Accuracy:  {accuracy*100:.2f}%")
print(f"Precision: {precision*100:.2f}%")
print(f"Recall:    {recall*100:.2f}%")
print(f"F1-Score:  {f1*100:.2f}%")
print(f"{'='*50}")

In [ ]:
# Classification Report
print("\nClassification Report:")
print("="*70)
print(classification_report(y_true, y_pred, target_names=CLASSES, digits=4))
print("="*70)

# Confusion Matrix
cm = confusion_matrix(y_true, y_pred)

# Plot confusion matrix
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=CLASSES, yticklabels=CLASSES, cbar=True)
plt.title('Confusion Matrix - Brain Tumor Detection', fontsize=14, fontweight='bold')
plt.ylabel('True Label', fontsize=12)
plt.xlabel('Predicted Label', fontsize=12)
plt.tight_layout()
plt.savefig('confusion_matrix.png', dpi=300, bbox_inches='tight')
plt.show()

print("Confusion matrix plot saved as 'confusion_matrix.png'")

## Section 7: Save the Trained Model

### Model Saving Options
1. **H5 Format**: Keras native format (used in main application)
2. **SavedModel Format**: TensorFlow's recommended format
3. **ONNX Format**: For cross-platform compatibility (optional)

In [ ]:
# Save the trained model in H5 format (compatible with Flask app)
output_path = "../static/models/model_2.h5"

# Create directory if it doesn't exist
os.makedirs(os.path.dirname(output_path), exist_ok=True)

# Save model
model.save(output_path)
print(f"✓ Model saved to: {output_path}")

# Verify model was saved
if os.path.exists(output_path):
    file_size = os.path.getsize(output_path) / (1024 * 1024)  # Size in MB
    print(f"✓ File size: {file_size:.2f} MB")
else:
    print(f"✗ Error: Model file not found at {output_path}")

In [ ]:
# Optional: Save model in SavedModel format (TensorFlow format)
savedmodel_path = "../static/models/brain_tumor_savedmodel"

model.save(savedmodel_path)
print(f"✓ Model also saved in SavedModel format: {savedmodel_path}")

# Optional: Save model information and metrics
model_info = {
    'model_name': 'Brain Tumor Detection CNN',
    'input_shape': (IMG_SIZE, IMG_SIZE, 3),
    'output_classes': NUM_CLASSES,
    'classes': CLASSES,
    'test_accuracy': float(test_accuracy),
    'test_precision': float(precision),
    'test_recall': float(recall),
    'test_f1_score': float(f1),
    'total_parameters': model.count_params(),
    'training_date': pd.Timestamp.now().isoformat()
}

# Save model info as JSON
import json
with open('model_info.json', 'w') as f:
    json.dump(model_info, f, indent=4)

print("\n✓ Model information saved to: model_info.json")
print("\nModel Summary:")
print(json.dumps(model_info, indent=2))

## Next Steps After Training

### 1. **Load Trained Model for Inference**
```python
from tensorflow.keras.models import load_model

# Load the trained model
model = load_model('model_2.h5')

# Make predictions on new images
predictions = model.predict(new_images)
```

### 2. **Integrate with Flask Application**
- Copy `model_2.h5` to `../static/models/` directory
- The Flask app will automatically load the model on first request

### 3. **Model Deployment**
- Use the saved model for production inference
- Consider containerization with Docker
- Deploy on cloud platforms (AWS, Google Cloud, Azure)

### 4. **Model Monitoring & Retraining**
- Monitor model performance on new data
- Retrain periodically with fresh data
- Track metrics over time

### 5. **Further Improvements**
- **Transfer Learning**: Use pre-trained models (VGG16, ResNet, Inception)
- **Ensemble Methods**: Combine multiple models for better accuracy
- **Data Augmentation**: Increase training data with augmentation techniques
- **Hyperparameter Tuning**: Use GridSearchCV or Bayesian Optimization

---

## Training Tips and Best Practices

### 1. **Data Quality**
- Ensure balanced class distribution
- Remove corrupted or low-quality images
- Normalize pixel values (0-1 range)

### 2. **Model Architecture**
- Use Batch Normalization for stability
- Apply Dropout for regularization
- Use appropriate activation functions

### 3. **Training Configuration**
- Use validation data to monitor overfitting
- Implement early stopping
- Use learning rate scheduling
- Apply L1/L2 regularization if needed

### 4. **Hyperparameter Tuning**
- Learning rate: 0.0001 to 0.01
- Batch size: 16, 32, or 64
- Dropout rate: 0.3 to 0.5
- Optimizer: Adam, SGD with momentum

### 5. **Handling Imbalanced Data**
```python
# Calculate class weights
from sklearn.utils.class_weight import compute_class_weight

class_weights = compute_class_weight(
    'balanced',
    classes=np.unique(y_train),
    y=y_train
)

# Use in training
model.fit(X_train, y_train, 
          class_weight=dict(enumerate(class_weights)))
```

---

## Troubleshooting Guide

| Issue | Solution |
|-------|----------|
| **Out of Memory (OOM)** | Reduce batch size, use gradient accumulation |
| **Overfitting** | Increase dropout, use regularization, more data augmentation |
| **Underfitting** | Increase model complexity, reduce dropout, train longer |
| **Slow Training** | Use GPU acceleration, reduce image size, smaller batch size |
| **Poor Accuracy** | Check data quality, adjust hyperparameters, increase training data |
| **NaN Values** | Check learning rate, normalize data, check for bad data |

---

## Resources

- [TensorFlow Documentation](https://www.tensorflow.org/docs)
- [Keras API Reference](https://keras.io/api/)
- [Deep Learning Best Practices](https://cs231n.github.io/)
- [Kaggle Brain Tumor Dataset](https://www.kaggle.com/datasets/ahmedhamada0/brain-tumor-detection)

---

**Training Complete!** 🎉

Your model is now ready for inference in the Flask web application.
Replace the model file in `static/models/model_2.h5` with your newly trained model.